In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import  cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

###Data loading

In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

In [0]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

###Dummy model

In [0]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor().fit(X,y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [0]:
[x for x in df.columns if "price" in x]

['price_currency', 'price_details', 'price_value']

In [0]:
df.price_currency.value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[df.price_currency != 'EUR']
df.shape

(106290, 155)

###Features

In [0]:
[print(x) for x in df.columns]

breadcrumb
created_at
price_currency
price_details
price_value
seller_address
seller_name
seller_type
feature_czujniki-parkowania-przednie
feature_poduszka-powietrzna-chroniąca-kolana
feature_kurtyny-powietrzne
feature_klimatyzacja-dwustrefowa
feature_światła-led
feature_czujnik-zmierzchu
feature_elektrycznie-ustawiane-lusterka
feature_asr-(kontrola-trakcji)
feature_poduszka-powietrzna-kierowcy
feature_cd
feature_elektryczne-szyby-przednie
feature_poduszka-powietrzna-pasażera
feature_system-start-stop
feature_światła-do-jazdy-dziennej
feature_komputer-pokładowy
feature_elektryczne-szyby-tylne
feature_klimatyzacja-manualna
feature_tapicerka-welurowa
feature_czujnik-deszczu
feature_światła-przeciwmgielne
feature_ogrzewanie-postojowe
feature_radio-niefabryczne
feature_regulowane-zawieszenie
feature_ogranicznik-prędkości
feature_zmieniarka-cd
feature_szyberdach
feature_isofix
feature_asystent-pasa-ruchu
feature_alufelgi
feature_bluetooth
feature_nawigacja-gps
feature_asystent-parkowania
fe

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [0]:
df.param_color.factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
SUFFIX_CAT = '_cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_vals = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
    df[feat] = factorized_vals 
  else:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
len(cat_feats)

154

In [0]:
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df.price_value.values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv =3,scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [0]:
m = DecisionTreeRegressor(max_depth=5).fit(X,y)
imp = PermutationImportance(m, random_state =0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd_cat
0.2008 ± 0.0047,param_faktura-vat_cat
0.1943 ± 0.0088,param_stan_cat
0.1423 ± 0.0086,param_rok-produkcji_cat
0.0629 ± 0.0047,param_moc_cat
0.0424 ± 0.0014,feature_kamera-cofania_cat
0.0412 ± 0.0008,param_skrzynia-biegów_cat
0.0286 ± 0.0037,param_marka-pojazdu_cat
0.0191 ± 0.0022,param_pojemność-skokowa_cat
0.0163 ± 0.0005,feature_bluetooth_cat
